In [45]:
import tensorflow as tf
from ipynb.fs.defs.SentimentAnalysis_Lexicon import create_feature_sets_and_labels
import pickle
import numpy as np

In [46]:
with open('sentiment_set.pickle', 'rb') as file:
    data = pickle.load(file)

In [47]:
(train_x, train_y, test_x, test_y) = (data[0], data[1], data[2], data[3])

In [48]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

In [49]:
n_classes = 2
batch_size = 100

In [50]:
x = tf.placeholder('float', [None, len(train_x[0])])
y = tf.placeholder('float')

In [51]:
def neural_network_model(data):
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

In [52]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                epoch_loss += c
                
                i += batch_size
                
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_x, y: test_y}))

In [53]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 240207.140625
Epoch 1 completed out of 10 loss: 110861.82794189453
Epoch 2 completed out of 10 loss: 63851.66241455078
Epoch 3 completed out of 10 loss: 36854.64491271973
Epoch 4 completed out of 10 loss: 21806.14531326294
Epoch 5 completed out of 10 loss: 17444.6316947937
Epoch 6 completed out of 10 loss: 21717.803176879883
Epoch 7 completed out of 10 loss: 14083.859797477722
Epoch 8 completed out of 10 loss: 12654.226956367493
Epoch 9 completed out of 10 loss: 11053.862113714218
Accuracy: 0.89735305
